## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


### (Advanced Users) Installing additional dependencies

Here we show you how to install additional packages into this Deepnote project. We will use the package [gcsopt](https://pypi.org/project/gcsopt/) as an example. To install additional dependencies, you have two choices:
- **Recommended** (automatically installs your dependencies every time you boot the Deepnote machine):
  1. Add the dependency to the list of dependencies in `reqirements.txt` on the left.  
      - **Note**: Make sure to NOT remove the text that is already in the file. You should add your dependencies on new lines AFTER `manipulation=year.month.day`. For our example, the file contents will look like:
          ```
          --extra-index-url https://drake-packages.csail.mit.edu/whl/nightly
          manipulation==2025.9.23
          gcsopt
          ```
  2. Restart the Deepnote machine (or run the `Init` notebook)
- **Alternatively:** Run `!pip install gcsopt` within a cell in the Deepnote notebook. You will have to do this everytime you restart your Deepnote machine.

### (Advanced Users) Updating the `manipulation` version
We continually add new features to the `manipulation` package, which is automatically updated in Deepnote. However, after you first click "Duplicate", you will stop receiving these updates. Follow these steps to update your version to the latest version:
1. Find the latest version number for `manipulation` on the [PyPi package page](https://pypi.org/project/manipulation/#history).
2. Update the version number for `manipulation=year.month.day` in the `requirements.txt` file on the left
3. Restart the Deepnote machine (or run the `Init` notebook)

The changes will persist every time you start your Deepnote machine.

Good luck!

---


## Placeholder code: iiwa Simulation

As a starting point, the code below sets up a simple simulation with a single iiwa, visualized in Meshcat.

### Setup and imports
Let us first get our imports out of the way:

In [1]:
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    BasicVector,
    LeafSystem,
    InverseKinematics,
    SnoptSolver,
    RotationMatrix,
    RigidTransform,
    Solve,
    LogVectorOutput,
    PiecewisePolynomial,
    TrajectorySource,
    ImageWriter,
    PixelType,
    JacobianWrtVariable,
    Context,
    MultibodyPlant,
    Integrator,
    ConstantVectorSource
)

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.geometry import Cylinder, Rgba

import matplotlib.pyplot as plt
import numpy as np
import pygame
import math
import os
import h5py
import numpy as np
import collections

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [2]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


### Setting up the Scenario

In [ ]:
import os
import numpy as np
from pathlib import Path

# Drawer configuration (from predefined_grasps.ipynb)
DRAWER_CONFIGS = {
    "label53": {"handle_label": "handle5", "r_min": 0.3, "r_max": 0.335, "lambda": 0.00004, "kp_null": 1},
    "label62": {"handle_label": "handle3", "r_min": 0.4, "r_max": 0.45, "lambda": 0.004, "kp_null": 5},
    "label31": {"handle_label": "handle3", "r_min": 0.39, "r_max": 0.43, "lambda": 0.0004, "kp_null": 1.5}
}

def calculate_drawer_pose(drawer_name, fov_deg=0, face_noise_deg=0, cabinet_depth=1.0, cabinet_width=1.0):
    """
    Calculate drawer position using the same logic as predefined_grasps.ipynb.
    
    Args:
        drawer_name: Name of the drawer (e.g., "label31")
        fov_deg: Field of view in degrees (0 = exact 270° placement)
        face_noise_deg: Face orientation noise in degrees (0 = perfect alignment)
        cabinet_depth: Depth of cabinet in meters
        cabinet_width: Width of cabinet in meters
        
    Returns (x, y, yaw_degrees)
    """
    r_min, r_max = DRAWER_CONFIGS[drawer_name]["r_min"], DRAWER_CONFIGS[drawer_name]["r_max"]
    r = (r_min + r_max) / 2.0  # Use midpoint of range for consistent placement
    
    # Calculate theta with fov_deg=0 for exact 270° placement
    theta_rad = np.radians(270 - fov_deg/2 + fov_deg/2)  # Simplifies to 270° when fov_deg=0
    
    x = r * np.cos(theta_rad)
    y = r * np.sin(theta_rad)
    
    # Adjust angle since drawer origin is bottom right corner of cabinet
    perfect_yaw_rad = np.arctan2(-y, -x)
    
    # Add face noise (0 when face_noise_deg=0)
    yaw_rad = perfect_yaw_rad + np.radians(face_noise_deg/2 - face_noise_deg/2)  # Simplifies to perfect_yaw_rad
    
    local_center_x = cabinet_depth / 2.0
    local_center_y = cabinet_width / 2.0
    
    c_yaw = np.cos(yaw_rad)
    s_yaw = np.sin(yaw_rad)
    
    world_offset_x = local_center_x * c_yaw - local_center_y * s_yaw
    world_offset_y = local_center_x * s_yaw + local_center_y * c_yaw
    
    final_origin_x = x - world_offset_x
    final_origin_y = y - world_offset_y
    
    return final_origin_x, final_origin_y, np.degrees(yaw_rad)

# Get the URDF path and position for label31 drawer
drawer_name = "label31"
drawer_urdf_path = os.path.join(Path.cwd(), "urdf/custom/output", f"{drawer_name}.urdf")

# Calculate drawer pose with fov_deg=0 and face_noise_deg=0 for exact positioning
x, y, yaw = calculate_drawer_pose(drawer_name, fov_deg=0, face_noise_deg=0)

print(f"Drawer position: x={x:.4f}, y={y:.4f}, yaw={yaw:.4f}°")

scenario_string = f"""directives:
# add robot
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0

# add gripper
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}

# add camera mounted to world
- add_frame:
    name: camera0_origin
    X_PF:
        base_frame: world
        rotation: !Rpy {{ deg: [250, 0, 180.0]}}
        translation: [0, 2, 1.4]
- add_model:
    name: camera0
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera0_origin
    child: camera0::base

# add camera mounted to robot wrist
- add_frame:
    name: camera_wrist
    X_PF:
      base_frame: iiwa::iiwa_link_7
      translation: [-0.05, 0, 0.07]
      rotation: !Rpy {{deg: [0, 0, -90]}}
- add_model:
    name: camera1
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: iiwa::camera_wrist
    child: camera1::base

# add camera mounted to world pointing top down
- add_frame:
    name: camera2_origin
    X_PF:
        base_frame: world
        rotation: !Rpy {{ deg: [0, 180.0, 0.0]}}
        translation: [0, -0.5, 3.0]
- add_model:
    name: camera2
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera2_origin
    child: camera2::base

# add drawer (label31) - fov_deg=0, face_noise_deg=0
- add_model:
    name: drawer
    file: file://{drawer_urdf_path}
- add_frame:
    name: drawer_origin
    X_PF:
      base_frame: world
      translation: [{x}, {y}, 0]
      rotation: !Rpy {{ deg: [0, 0, {yaw}]}}
- add_weld:
    parent: drawer_origin
    child: drawer::base_link

cameras:
  camera0:
    name: camera0
    depth: True
    X_PB:
      base_frame: camera0::base
  camera1:
    name: camera1
    depth: True
    focal: !FovDegrees
        x: 110   # horizontal FOV in degrees
    X_PB:
      base_frame: camera1::base
  camera2:
    name: camera2
    depth: True
    X_PB:
      base_frame: camera2::base

model_drivers:
  iiwa: !IiwaDriver
    control_mode: position_only
    hand_model_name: wsg
  wsg: !SchunkWsgDriver {{}}
"""

### Teleop with Spacemouse

You need to download the spacemouse driver and spacenavd package, and make spacenavd active in command line with `systemctl start spacenavd`. Otherwise, this code will not detect the spacemouse.

In [12]:
import ctypes
import numpy as np

# -------- Spacenav / SpaceMouse bindings via libspnav --------

libspnav = ctypes.CDLL("libspnav.so")

SPNAV_EVENT_MOTION = 1
SPNAV_EVENT_BUTTON = 2

class SpnavMotion(ctypes.Structure):
    _fields_ = [
        ("x", ctypes.c_int),
        ("y", ctypes.c_int),
        ("z", ctypes.c_int),
        ("rx", ctypes.c_int),
        ("ry", ctypes.c_int),
        ("rz", ctypes.c_int),
    ]

class SpnavButton(ctypes.Structure):
    _fields_ = [
        ("bnum", ctypes.c_int),
        ("press", ctypes.c_int),
    ]

class SpnavEvent(ctypes.Structure):
    _fields_ = [
        ("type", ctypes.c_int),
        ("motion", SpnavMotion),
        ("button", SpnavButton),
    ]


class SpaceMouse:
    """Thin wrapper around libspnav using ctypes."""

    def __init__(self):
        rc = libspnav.spnav_open()
        if rc == -1:
            print("ERROR: Could not connect to spacenavd. Is it running?")
            self.active = False
        else:
            print("SpaceMouse connected via libspnav!")
            self.active = True

        # Reusable event struct to avoid reallocations
        self._event = SpnavEvent()

    def poll_event(self):
        """Non-blocking poll. Returns SpnavEvent or None."""
        if not self.active:
            return None
        n = libspnav.spnav_poll_event(ctypes.byref(self._event))
        if n > 0:
            # copy to avoid overwriting by next poll
            ev_copy = SpnavEvent()
            ctypes.pointer(ev_copy)[0] = self._event
            return ev_copy
        return None

    def close(self):
        if self.active:
            libspnav.spnav_close()
            self.active = False


In [13]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant) -> None:
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context: Context, output: BasicVector) -> None:
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kV,
            self._G,
            [0, 0, 0],
            self._W,
            self._W,
        )
        J_G = J_G[:, self.iiwa_start : self.iiwa_end + 1]  # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G)
        # if v[0] != 0:
        #     print("velocity command: ", v)
        output.SetFromVector(v)

In [36]:
class TeleopController(LeafSystem):
    """
    Buttons for end-effector pose to joint positions via IK, using SpaceMouse.
    """
    def __init__(self, meshcat, plant):
        LeafSystem.__init__(self)
        
        # --- Spacemouse initialization ---
        self.spacemouse = SpaceMouse()
        self.spacemouse_active = self.spacemouse.active

        # Scale factors
        self.TRANSLATION_GAIN = 0.001     # meters per unit input
        self.ROTATION_GAIN    = 0.001     # rad per unit input
        
        self.DEADZONE_THRESHOLD = 1e-7  # Ignore movements smaller than this

        self.last_twist = np.zeros(6)
        self.last_press = 0.1

        # Register output ports
        self.DeclareVectorOutputPort(
            "V_WG_cmd",
            BasicVector(6),
            self.DoCalcOutput
        )
        self.DeclareVectorOutputPort(
            "wsg_cmd",
            BasicVector(1),
            lambda context, output: output.SetFromVector([self.last_press])
        )

        meshcat.AddButton("Stop Simulation")


    def read_spacemouse(self):
        """
        Returns a filtered 6D twist vector [vx, vy, vz, wx, wy, wz]
        Only returns non-zero values when there's actual new input.
        """
        if not self.spacemouse_active:
            return np.zeros(6)

        event = self.spacemouse.poll_event()
        if event is not None:
            if event.type == SPNAV_EVENT_MOTION:
                twist_raw = np.array([
                    self.TRANSLATION_GAIN * event.motion.x,
                    self.TRANSLATION_GAIN * event.motion.y,
                    self.TRANSLATION_GAIN * event.motion.z,
                    self.ROTATION_GAIN * event.motion.rx,
                    self.ROTATION_GAIN * event.motion.ry,
                    self.ROTATION_GAIN * event.motion.rz
                ])

                # Check if input is significant (deadzone)
                if np.linalg.norm(twist_raw) > self.DEADZONE_THRESHOLD:
                    self.last_twist = twist_raw
            elif event.type == SPNAV_EVENT_BUTTON:
                self.last_press = 0.1 if self.last_press == 0 else 0
        else:
            # if no spacemouse event, decay twist to prevent drift
            self.last_twist *= 0.9

        return self.last_twist


    def DoCalcOutput(self, context, output):
        """
        Outputs the latest calculated joint position command.
        """
        twist = self.read_spacemouse()
        output.SetFromVector(twist)


In [37]:
WSG_MIN = 0.0        # fully closed width (meters)
WSG_MAX = 0.1        # fully open width (meters)

def normalize_wsg(width):
    return np.clip((width - WSG_MIN) / (WSG_MAX - WSG_MIN), 0.0, 1.0)

def unnormalize_wsg(norm_width):
    return WSG_MIN + norm_width * (WSG_MAX - WSG_MIN)

WSG_MAX_SPEED = 2.8  # meters/sec width change

def normalize_wsg_velocity(width_velocity):
    v = width_velocity / WSG_MAX_SPEED
    return np.clip(v, -1.0, 1.0)

Run simulation to collect teleoperation data and save it to files.

In [ ]:
target_dt = 0.05  # 20 Hz

scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant") 

camera_world = station.GetSubsystemByName("rgbd_sensor_camera0")
camera_wrist = station.GetSubsystemByName("rgbd_sensor_camera1")
camera_top = station.GetSubsystemByName("rgbd_sensor_camera2")

# add controllers
teleop = builder.AddSystem(TeleopController(meshcat, plant))
controller = builder.AddSystem(PseudoInverseController(plant))
integrator = builder.AddSystem(Integrator(7))

# Connect controller outputs to station inputs
builder.Connect(
    teleop.GetOutputPort("V_WG_cmd"),
    controller.GetInputPort("V_WG")
)
builder.Connect(
    controller.get_output_port(), 
    integrator.get_input_port()
)
builder.Connect(
    integrator.get_output_port(), 
    station.GetInputPort("iiwa.position")
)
builder.Connect(
    station.GetOutputPort("iiwa.position_measured"),
    controller.GetInputPort("iiwa.position"),
)
builder.Connect(
    teleop.GetOutputPort("wsg_cmd"),
    station.GetInputPort("wsg.position")
)


diagram = builder.Build()
context = diagram.CreateDefaultContext()
station_context = diagram.GetSubsystemContext(station, context)

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

sim_context = simulator.get_mutable_context()
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(sim_context),
    plant.GetPositions(
        plant.GetMyContextFromRoot(sim_context),
        plant.GetModelInstanceByName("iiwa"),
    ),
)
diagram.ForcedPublish(sim_context)

print("\n--- Starting Cartesian Teleop Demo ---")
print("Please focus the Meshcat viewer window to enable keyboard input.")


image_data_camera0 = []
image_data_camera1 = []
image_data_camera2 = []
timestamps = []

qpos_list = []
qvel_list = []
action_list = []  # iiwa + gripper commands


# Run until the user clicks the "Stop Simulation" button in Meshcat
try:
    while meshcat.GetButtonClicks('Stop Simulation') == 0:
        
        simulator.AdvanceTo(simulator.get_context().get_time() + target_dt)

        sim_context = simulator.get_context()
        station_context = diagram.GetSubsystemContext(station, sim_context)

        # ---------- get plant state (qpos, qvel) ----------
        plant_context = station.GetSubsystemContext(plant, station_context)
        
        iiwa_model = plant.GetModelInstanceByName("iiwa")
        wsg_model  = plant.GetModelInstanceByName("wsg")

        # iiwa joints
        q_iiwa = plant.GetPositions(plant_context, iiwa_model)
        v_iiwa = plant.GetVelocities(plant_context, iiwa_model)

        # gripper joints
        q_wsg = plant.GetPositions(plant_context, wsg_model)
        v_wsg = plant.GetVelocities(plant_context, wsg_model)
        wsg_width = normalize_wsg(q_wsg[1] - q_wsg[0])
        wsg_vel = normalize_wsg_velocity(v_wsg[1] - v_wsg[0])

        # concatenate to get exactly (7 + 1,) vectors
        qpos = np.concatenate([q_iiwa, [wsg_width]], axis=0).copy()
        qvel = np.concatenate([v_iiwa, [wsg_vel]], axis=0).copy()

        qpos_list.append(qpos)
        qvel_list.append(qvel)

        # ---------- get current action ----------
        controller_context = diagram.GetSubsystemContext(controller, sim_context)
        integrator_context = diagram.GetSubsystemContext(integrator, sim_context)
        teleop_context = diagram.GetSubsystemContext(teleop, sim_context)

        iiwa_cmd = integrator.get_output_port().Eval(integrator_context)
        wsg_cmd = teleop.GetOutputPort("wsg_cmd").Eval(teleop_context)
        # normalize gripper commands
        wsg_cmd = [normalize_wsg(x) for x in wsg_cmd]

        # flatten to 1D arrays and concatenate
        iiwa_cmd_np = np.asarray(iiwa_cmd).ravel()
        wsg_cmd_np = np.asarray(wsg_cmd).ravel()
        action = np.concatenate([iiwa_cmd_np, wsg_cmd_np], axis=0)
        action_list.append(action)

        # Grab the current images from the output ports
        img0 = station.GetOutputPort("camera0.rgb_image").Eval(station_context)  # ImageRgba8U object
        img1 = station.GetOutputPort("camera1.rgb_image").Eval(station_context)  # ImageRgba8U object
        img2 = station.GetOutputPort("camera2.rgb_image").Eval(station_context)  # ImageRgba8U object

        # Convert to numpy arrays for easier handling
        np_img0 = np.array(np.copy(img0.data)).reshape(img0.height(), img0.width(), -1)  # RGBA
        np_img1 = np.array(np.copy(img1.data)).reshape(img1.height(), img1.width(), -1)
        np_img2 = np.array(np.copy(img2.data)).reshape(img2.height(), img2.width(), -1)
        
        # keep only RGB channels; ACT expects (H, W, 3) uint8 images
        image_data_camera0.append(np_img0[:, :, :3].copy())
        image_data_camera1.append(np_img1[:, :, :3].copy())
        image_data_camera2.append(np_img2[:, :, :3].copy())

        timestamps.append(sim_context.get_time())
finally:
    teleop.spacemouse.close()

print("Simulation stopped by user.")

SpaceMouse connected via libspnav!

--- Starting Cartesian Teleop Demo ---
Please focus the Meshcat viewer window to enable keyboard input.
Simulation stopped by user.


In [ ]:
# ---------- convert buffers to numpy arrays ----------
qpos_arr = np.stack(qpos_list, axis=0).astype(np.float32)      # (T, nq)
qvel_arr = np.stack(qvel_list, axis=0).astype(np.float32)      # (T, nq)
action_arr = np.stack(action_list, axis=0).astype(np.float32)  # (T, act_dim)

images0_arr = np.stack(image_data_camera0, axis=0).astype(np.uint8)  # (T, H, W, 3)
images1_arr = np.stack(image_data_camera1, axis=0).astype(np.uint8)  # (T, H, W, 3)
images2_arr = np.stack(image_data_camera2, axis=0).astype(np.uint8)  # (T, H, W, 3)

timestamps_arr = np.array(timestamps, dtype=np.float64)

# print("qpos: ", qpos_arr, qpos_arr.shape)
# print("qvel: ", qvel_arr, qvel_arr.shape)
# print("actions: ", action_arr, action_arr.shape)
# print(timestamps_arr)

# ---------- write ACT-style HDF5 episode ----------
episode_idx = 2  # you can change this per demonstration
h5_path = os.path.join("teleop_data/sim_open_drawer", f"episode_{episode_idx}.hdf5")

with h5py.File(h5_path, "w") as root:
    # mark this as simulation data (ACT uses this flag)
    root.attrs["sim"] = True

    # actions: shape (T, act_dim)
    root.create_dataset("action", data=action_arr, compression="gzip")

    # observations group
    obs_grp = root.create_group("observations")
    obs_grp.create_dataset("qpos", data=qpos_arr, compression="gzip")
    obs_grp.create_dataset("qvel", data=qvel_arr, compression="gzip")

    # optional: store timestamps (not strictly required by ACT but often useful)
    obs_grp.create_dataset("time", data=timestamps_arr)

    # images subgroup, one dataset per camera
    img_grp = obs_grp.create_group("images")
    img_grp.create_dataset("camera0", data=images0_arr, compression="gzip")
    img_grp.create_dataset("camera1", data=images1_arr, compression="gzip")
    img_grp.create_dataset("camera2", data=images2_arr, compression="gzip")

print(f"Wrote ACT-style episode to {h5_path}")

Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_2.hdf5


Replay collected joint and gripper data. (Need to restart kernel, run first three cells to import dependencies and define scenario_string and meshcat, then run this cell.)

In [6]:
joint_data = np.load("downsampled_teleop_joint_data.npz")
joint_t_uniform = joint_data["time"]
joint_q_uniform = joint_data["q"]

gripper_data = np.load("downsampled_teleop_gripper_data.npz")
gripper_t_uniform = gripper_data["time"]
gripper_q_uniform = gripper_data["q"]

scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
plant = station.GetSubsystemByName("plant") 

traj_iiwa = PiecewisePolynomial.FirstOrderHold(joint_t_uniform, joint_q_uniform)
traj_wsg = PiecewisePolynomial.FirstOrderHold(gripper_t_uniform, gripper_q_uniform)

builder = DiagramBuilder()
builder.AddSystem(station)  # your station system already created

# Create trajectory sources
iiwa_source = builder.AddSystem(TrajectorySource(traj_iiwa))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg))

# Connect to the plant
builder.Connect(iiwa_source.get_output_port(0),
                station.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(0),
                station.GetInputPort("wsg.position"))

diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

# Run for the duration of the trajectory
simulator.AdvanceTo(joint_t_uniform[-1])

print("Replay finished!")

Replay finished!


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf4fb7c2-18e7-48b8-93e7-63f0209e538b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>